In [1]:
# import modules
import uproot, sys, time, math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
from scipy.special import betainc
from scipy.stats import norm

# import config functions
from jet_faking_plot_config import getWeight, zbi, sample_dict, getVarDict
from plot_var import variables, variables_data, ntuple_names, ntuple_names_BDT


# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 14.0,10.0  # Roughly 11 cm wde by 8 cm high
mpl.rcParams['font.size'] = 20.0 # Use 14 point font
sns.set(style="whitegrid")

font_size = {
    "xlabel": 17,
    "ylabel": 17,
    "xticks": 15,
    "yticks": 15,
    "legend": 14
}

plt.rcParams.update({
    "axes.labelsize": font_size["xlabel"],  # X and Y axis labels
    "xtick.labelsize": font_size["xticks"],  # X ticks
    "ytick.labelsize": font_size["yticks"],  # Y ticks
    "legend.fontsize": font_size["legend"]  # Legend
})

In [ ]:
tot = []
data = pd.DataFrame()
unweighted_bcut, weighted_bcut, unweighted_acut, weighted_acut = [], [], [], []
ntuple_names = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct', 'data23']

def test(fb):
    # checking if there are any none values
    mask = ak.is_none(fb['met_tst_et'])
    n_none = ak.sum(mask)
    print("Number of none values: ", n_none)
    # if n_none > 0:
    #     fb = fb[~mask]
    # print("Events after removing none values: ", len(fb), ak.sum(ak.is_none(fb['met_tst_et'])))

def print_cut(ntuple_name, fb, label):
    print(f"Unweighted Events {label}: ", len(fb))
    if ntuple_name == 'data23':
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name, jet_faking=True)))
    else: 
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name)))

for i in range(len(ntuple_names)):
    ucut, wcut = [], []
    start_time = time.time()
    ntuple_name = ntuple_names[i]
    if ntuple_name == 'data23': # data
        path = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/data23_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables_data, library="ak")
        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
                
        mask1 = (ak.firsts(fb['ph_topoetcone40'])-2450.)/ak.firsts(fb['ph_pt']) < 0.1   # jet_faking_photon cut
        fb = fb[mask1]
        fb = fb[fb['n_ph_baseline'] == 1]

    else: # MC
        path = f"/data/tmathew/ntups/mc23d/{ntuple_name}_y.root" 
        path_BDT = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/mc23d_{ntuple_name}_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables, library="ak")

        # add BDT score to fb
        f_BDT = uproot.open(path_BDT)['nominal']
        fb_BDT = f_BDT.arrays(["event", "BDTScore"], library="ak")
        tmp = fb["event"] == fb_BDT["event"]
        if np.all(tmp) == True:
            fb["BDTScore"] = fb_BDT["BDTScore"]
        else: 
            print("Something is wrong, need arranging")

        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
        fb = fb[fb['n_ph'] == 1]

    

    # Zjets and Wjets (rule out everything except for e->gamma)
    if ntuple_name == 'Zjets' or ntuple_name == 'Wjets':
        mask1 = ak.firsts(fb['ph_truth_type']) == 2
        mask2 = ak.firsts(fb['ph_truth_type']) == 2
        fb = fb[mask1 & mask2]
    
    print_cut(ntuple_name, fb, 'before cut')


    fb = fb[fb['n_mu_baseline'] == 0]
    fb = fb[fb['n_el_baseline'] == 0]
    fb = fb[fb['n_tau_baseline'] == 0]
    fb = fb[fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M']==1]
    fb = fb[ak.num(fb['ph_pt']) > 0] # prevent none values in Tbranch
    fb = fb[fb['met_tst_et'] >= 100000] # MET cut (basic cut)
    fb = fb[ak.firsts(fb['ph_pt']) >= 50000] # ph_pt cut (basic cut)
    fb = fb[fb['n_jet_central'] <= 4] # n_jet_central cut (basic cut)
    # goodPV on signal only
    if ntuple_name == 'ggHyyd':
        fb = fb[ak.num(fb['pv_z']) > 0]
        good_pv_tmp = (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5)
        fb = fb[good_pv_tmp]

    mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
                            (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    mask1 = mt_tmp >= 100 # trigger cut
    fb = fb[mask1]

    fb = fb[fb['BDTScore'] >= 0.1] # added cut 1

    print_cut(ntuple_name, fb, 'after basic cut')

    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # metsig_tmp = fb['met_tst_sig'] # added cut 2 
    # mask1 = metsig_tmp >= 7
    # fb = fb[mask1]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # dphi_met_phterm_tmp = np.arccos(np.cos(fb['met_tst_phi'] - fb['met_phterm_phi'])) # added cut 3
    # fb = fb[dphi_met_phterm_tmp >= 1.35]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # dmet_tmp = fb['met_tst_noJVT_et'] - fb['met_tst_et'] # added cut 4
    # mask1 = dmet_tmp >= -20000
    # mask2 = dmet_tmp <= 50000
    # fb = fb[mask1 * mask2]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # dphi_met_jetterm_tmp = np.where(fb['met_jetterm_et'] != 0,   # added cut 5
    #                         np.arccos(np.cos(fb['met_tst_phi'] - fb['met_jetterm_phi'])),
    #                         -999)
    # fb = fb[dphi_met_jetterm_tmp <= 0.7]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # ph_eta_tmp = np.abs(ak.firsts(fb['ph_eta'])) # added cut 6
    # fb = fb[ph_eta_tmp <= 1.75]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # phi1_tmp = ak.firsts(fb['jet_central_phi']) # added cut 7
    # phi2_tmp = ak.mask(fb['jet_central_phi'], ak.num(fb['jet_central_phi']) >= 2)[:, 1] 
    # dphi_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
    # dphi_jj_tmp = ak.fill_none(dphi_tmp, -999)
    # fb = fb[dphi_jj_tmp <= 2.5]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # jet_sum_tmp = ak.sum(fb['jet_central_pt'], axis=-1)
    # expr = (fb['met_tst_et'] + ak.firsts(fb['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
    # balance_tmp = ak.where(jet_sum_tmp != 0, expr, 999) 
    # fb = fb[balance_tmp >= 0.65]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))

    # mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
    #                     (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    # mask1 = mt_tmp >= 95
    # fb = fb[mask1]

    # print_cut(ntuple_name, fb)
    ucut.append(len(fb))
    if ntuple_name == 'data23':
        wcut.append(sum(getWeight(fb, ntuple_name, jet_faking=True)))
    else: 
        wcut.append(sum(getWeight(fb, ntuple_name)))


    unweighted_acut.append(ucut)
    weighted_acut.append(wcut)
    test(fb) # check for none value

    print(f"Reading Time for {ntuple_name}: {(time.time()-start_time)} seconds\n")



    tot.append(fb)

    fb = 0
    fb_BDT = 0
    tmp = 0


processing file:  /data/tmathew/ntups/mc23d/ggHyyd_y.root
Unweighted Events before cut:  195671
Weighted Events before cut:  19979.121
Unweighted Events after basic cut:  3729
Weighted Events after basic cut:  375.93835
Number of none values:  0
Reading Time for ggHyyd: 2.382730007171631 seconds

processing file:  /data/tmathew/ntups/mc23d/Zjets_y.root
Unweighted Events before cut:  3242488


In [9]:
def sel(tot):
    tot2 = []
    for i in range(len(tot)):
        fb2 = tot[i]

        # jet_sum_tmp = ak.sum(fb2['jet_central_pt'], axis=-1)
        # expr = (fb2['met_tst_et'] + ak.firsts(fb2['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
        # balance_tmp = ak.where(jet_sum_tmp != 0, expr, -999)
        # mask1 = balance_tmp >= 0.65
        # mask2 = balance_tmp == -999
        # fb2 = fb2[mask1 | mask2]
        
        metsig_tmp = fb2['met_tst_sig'] 
        mask1 = metsig_tmp >= 7
        fb2 = fb2[mask1]
        # mask2 = metsig_tmp <= 13
        # fb2 = fb2[mask1 * mask2]
        
        # ph_eta_tmp = np.abs(ak.firsts(fb2['ph_eta']))
        # fb2 = fb2[ph_eta_tmp <= 1.75]

        dphi_met_phterm_tmp = np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_phterm_phi'])) # added cut 3
        fb2 = fb2[dphi_met_phterm_tmp >= 1.55]

        # dmet_tmp = fb2['met_tst_noJVT_et'] - fb2['met_tst_et']
        # mask1 = dmet_tmp >= -20000
        # mask2 = dmet_tmp <= 50000
        # fb2 = fb2[mask1 * mask2]

        # phi1_tmp = ak.firsts(fb2['jet_central_phi']) # added cut 7
        # phi2_tmp = ak.mask(fb2['jet_central_phi'], ak.num(fb2['jet_central_phi']) >= 2)[:, 1] 
        # dphi_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
        # dphi_jj_tmp = ak.fill_none(dphi_tmp, -999)
        # fb2 = fb2[dphi_jj_tmp <= 2.5]

        # dphi_met_jetterm_tmp = np.where(fb2['met_jetterm_et'] != 0,   # added cut 5
        #                     np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_jetterm_phi'])),
        #                     -999)
        # fb2 = fb2[dphi_met_jetterm_tmp <= 0.70]
        
        tot2.append(fb2)
    return tot2

tot2 = sel(tot)
# tot2 = tot

signal_name = 'ggHyyd'  # Define signal dataset
cut_name = 'metsig'

def getCutDict():
    cut_dict = {}

    cut_dict['BDTScore'] = {
        'lowercut': np.arange(0, 0.4+0.1, 0.1) # BDTScore > cut
    }
    cut_dict['balance'] = {
        'lowercut': np.arange(0, 1.5 + 0.05, 0.05), # balance > cut
        'uppercut': np.arange(5, 8 + 0.2, 0.2) # balance < cut
    }
    cut_dict['dmet'] = {
        'lowercut': np.arange(-30000, 0 + 5000, 5000), # dmet > cut
        'uppercut': np.arange(10000, 100000 + 5000, 5000), # -10000 < dmet < cut
    }
    cut_dict['dphi_jj'] = {
        'uppercut': np.arange(1, 3.1 + 0.1, 0.1) # dphi_jj < cut
    }
    # cut_dict['dphi_met_phterm_minus_dphi_met_jetterm'] = {
    #     'lowercut': np.arange(0, 1.5+0.05, 0.05),
    #     'uppercut': np.arange(1.5, 3.1+0.05, 0.05)
    # }
    cut_dict['dphi_met_jetterm'] = {
        'lowercut': np.arange(0, 1 + 0.05, 0.05), # dphi_met_jetterm > cut 
        'uppercut': np.arange(0.5, 2 + 0.05, 0.05), # dphi_met_jetterm < cut 
    }
    cut_dict['dphi_met_phterm'] = {
        'lowercut': np.arange(1, 2 + 0.05, 0.05), # dphi_met_phterm > cut
        'uppercut': np.arange(2, 3.1 + 0.1, 0.1), # dphi_met_phterm < cut
    }
    cut_dict['dphi_ph_centraljet1'] = {
        'lowercut': np.arange(0, 2.5 + 0.1, 0.1), # dphi_ph_centraljet1 > cut
        'uppercut': np.arange(1.5, 3.1 + 0.1, 0.1) # dphi_ph_centraljet1 < cut
    }
    cut_dict['dphi_phterm_jetterm'] = {
        'lowercut': np.arange(1, 2.5 + 0.05, 0.05), # dphi_phterm_jetterm > cut
        'uppercut': np.arange(2, 4 + 0.1, 0.1) # dphi_phterm_jetterm < cut
    }
    cut_dict['met'] = {
        'lowercut': np.arange(100000, 140000 + 5000, 5000),  # met > cut
        'uppercut': np.arange(140000, 300000 + 5000, 5000),  # met < cut
    }
    cut_dict['metsig'] = {
        'lowercut': np.arange(0, 10 + 1, 1), # metsig > cut
        'uppercut': np.arange(10, 30 + 1, 1), # metsig < cut 
    }
    cut_dict['mt'] = {
        'lowercut': np.arange(80, 130+5, 5), # mt > cut
        'uppercut': np.arange(120, 230+5, 5) # mt < cut
    }
    cut_dict['n_jet_central'] = {
        'uppercut': np.arange(0, 8+1, 1) # njet < cut
    }
    cut_dict['ph_eta'] = {
        'uppercut': np.arange(1, 2.5 + 0.05, 0.05), # ph_eta < cut
    }
    cut_dict['ph_pt'] = {
        'lowercut': np.arange(50000, 100000 + 5000, 5000),  # ph_pt > cut
        'uppercut': np.arange(100000, 300000 + 10000, 10000),  # ph_pt > cut
    }

    return cut_dict
cut_config = getCutDict()

def calculate_significance(cut_var, cut_type, cut_values):
    sig_simple_list = []
    sig_s_plus_b_list = []
    sig_s_plus_1p3b_list = []
    sig_binomial_list = []

    sigacc_simple_list = []
    sigacc_s_plus_b_list = []
    sigacc_s_plus_1p3b_list = []
    sigacc_binomial_list = []

    acceptance_values = []  # Store acceptance percentages

    for cut in cut_values:
        sig_after_cut = 0
        bkg_after_cut = []
        sig_events = 0
        
        for i in range(len(ntuple_names)):
            fb = tot2[i]
            process = ntuple_names[i]
            var_config = getVarDict(fb, process, var_name=cut_var)
            x = var_config[cut_var]['var']
            mask = x != -999 # Apply cut: Remove -999 values 
            x = x[mask]

            if process == signal_name:
                sig_events = getWeight(fb, process)
                sig_events = sig_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                sig_after_cut = ak.sum(sig_events[mask])
            
            else:
                bkg_events = getWeight(fb, process)
                bkg_events = bkg_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                bkg_after_cut.append(ak.sum(bkg_events[mask]))

       # Now compute different types of significance
        total_bkg = sum(bkg_after_cut)
        total_signal = sig_after_cut

        # Avoid zero division carefully
        if total_bkg > 0:
            sig_simple = total_signal / np.sqrt(total_bkg)
            sig_s_plus_b = total_signal / np.sqrt(total_signal + total_bkg) if (total_signal + total_bkg) > 0 else 0
            sig_s_plus_1p3b = total_signal / np.sqrt(total_signal + 1.3 * total_bkg) if (total_signal + 1.3*total_bkg) > 0 else 0
            sig_binomial = zbi(total_signal, total_bkg, sigma_b_frac=0.3)
        else:
            sig_simple = sig_s_plus_b = sig_s_plus_1p3b = sig_binomial = 0

        # Acceptance
        acceptance = total_signal / sum(sig_events) if sum(sig_events) > 0 else 0
        acceptance_values.append(acceptance * 100)  # percentage

        # Save significance
        sig_simple_list.append(sig_simple)
        sig_s_plus_b_list.append(sig_s_plus_b)
        sig_s_plus_1p3b_list.append(sig_s_plus_1p3b)
        sig_binomial_list.append(sig_binomial)

        # Save significance × acceptance
        sigacc_simple_list.append(sig_simple * acceptance)
        sigacc_s_plus_b_list.append(sig_s_plus_b * acceptance)
        sigacc_s_plus_1p3b_list.append(sig_s_plus_1p3b * acceptance)
        sigacc_binomial_list.append(sig_binomial * acceptance)

    return (sig_simple_list, sig_s_plus_b_list, sig_s_plus_1p3b_list, sig_binomial_list,
            sigacc_simple_list, sigacc_s_plus_b_list, sigacc_s_plus_1p3b_list, sigacc_binomial_list,
            acceptance_values)

# Compute significance for each variable dynamically
for cut_var, cut_types in cut_config.items():
    for cut_type, cut_values in cut_types.items():
        (sig_simple_list, sig_s_plus_b_list, sig_s_plus_1p3b_list, sig_binomial_list,
         sigacc_simple_list, sigacc_s_plus_b_list, sigacc_s_plus_1p3b_list, sigacc_binomial_list,
         acceptance_values) = calculate_significance(cut_var, cut_type, cut_values)

        # Plot results
        fig, (ax_top, ax_bot) = plt.subplots(2, 1, figsize=(8, 10), sharex=True)

        # Top plot: Significance vs. Cut
        ax_top.plot(cut_values, sig_simple_list, marker='o', label='S/√B')
        ax_top.plot(cut_values, sig_s_plus_b_list, marker='s', label='S/√(S+B)')
        ax_top.plot(cut_values, sig_s_plus_1p3b_list, marker='^', label='S/√(S+1.3B)')
        ax_top.plot(cut_values, sig_binomial_list, marker='x', label='BinomialExpZ')
        ax_top.set_ylabel('Significance')
        ax_top.set_title(f'Significance vs. {cut_var} ({cut_type})')
        ax_top.legend()
        ax_top.grid(True)

        # Bottom plot: Significance * Acceptance vs. Cut
        ax_bot.plot(cut_values, sigacc_simple_list, marker='o', label='(S/√B) × Acceptance')
        ax_bot.plot(cut_values, sigacc_s_plus_b_list, marker='s', label='(S/√(S+B)) × Acceptance')
        ax_bot.plot(cut_values, sigacc_s_plus_1p3b_list, marker='^', label='(S/√(S+1.3B)) × Acceptance')
        ax_bot.plot(cut_values, sigacc_binomial_list, marker='x', label='BinomialExpZ × Acceptance')

        for i, txt in enumerate(acceptance_values):
            ax_bot.text(cut_values[i], sigacc_simple_list[i], f'{txt:.1f}%', 
                        fontsize=10, ha='right', va='bottom', color='purple')
            
        ax_bot.set_xlabel(f'{cut_var} Cut')
        ax_bot.set_ylabel('Significance × Acceptance')
        ax_bot.set_title(f'Significance × Acceptance vs. {cut_var} ({cut_type})')
        
        ax_bot.set_xticks(cut_values)
        ax_bot.set_xticklabels(ax_bot.get_xticks(), rotation=45, ha='right')
        ax_bot.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        # ax_bot.xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))  # Show at most 10 x-ticks
        
        var_configs_tmp = getVarDict(tot2[0], signal_name, cut_var)
        ax_bot.set_xlabel(var_configs_tmp[cut_var]['title'])
        ax_bot.legend()
        ax_bot.grid(True)

        plt.tight_layout()
        plt.savefig(f"../jets_faking_photons/lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
        print(f"Successfully saved to ../jets_faking_photons/lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
        plt.close()

Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_BDTScore_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_balance_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_balance_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_dmet_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_dmet_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_dphi_jj_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_dphi_met_jetterm_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_dphi_met_jetterm_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_metsigcut/significance_dphi_met_phterm_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135

In [ ]:
# print out the image list in common_config.js
var_config = getVarDict(tot2[0], 'ggHyyd')

for var in var_config:

    print(f"'mc23d_balancecut/{var}_nodijet.png',")
    print(f"'mc23d_abs_weight_selectioncut/{var}_absWeightSelection.png',")

'mc23d_balancecut/vtx_sumPt_nodijet.png',
'mc23d_abs_weight_selectioncut/vtx_sumPt_absWeightSelection.png',
'mc23d_balancecut/n_ph_nodijet.png',
'mc23d_abs_weight_selectioncut/n_ph_absWeightSelection.png',
'mc23d_balancecut/n_ph_baseline_nodijet.png',
'mc23d_abs_weight_selectioncut/n_ph_baseline_absWeightSelection.png',
'mc23d_balancecut/n_el_baseline_nodijet.png',
'mc23d_abs_weight_selectioncut/n_el_baseline_absWeightSelection.png',
'mc23d_balancecut/n_mu_baseline_nodijet.png',
'mc23d_abs_weight_selectioncut/n_mu_baseline_absWeightSelection.png',
'mc23d_balancecut/n_tau_baseline_nodijet.png',
'mc23d_abs_weight_selectioncut/n_tau_baseline_absWeightSelection.png',
'mc23d_balancecut/puWeight_nodijet.png',
'mc23d_abs_weight_selectioncut/puWeight_absWeightSelection.png',
'mc23d_balancecut/actualIntPerXing_nodijet.png',
'mc23d_abs_weight_selectioncut/actualIntPerXing_absWeightSelection.png',
'mc23d_balancecut/mt_nodijet.png',
'mc23d_abs_weight_selectioncut/mt_absWeightSelection.png',
'mc23d

## Back-up

In [ ]:

    var = {
        'vtx_sumPt': ak.flatten(fb['vtx_sumPt']),
        'n_ph': fb['n_ph'],
        'n_ph_baseline': fb['n_ph_baseline'],
        'n_el_baseline': fb['n_el_baseline'],
        'n_mu_baseline': fb['n_mu_baseline'],
        'n_tau_baseline': fb['n_tau_baseline'],
        'puWeight': fb['pu_weight'],
        'actualIntPerXing': fb['actualIntPerXing'],
        'mt': np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000,
        'metsig': fb['met_tst_sig'],
        'metsigres': fb['met_tst_et'] / fb['met_tst_sig'],
        'met': fb['met_tst_et'],  # applying the '+50000' shift
        'met_noJVT': fb['met_tst_noJVT_et'],
        'met_cst': fb['met_cst_et'],
        'met_track': fb['met_track_et'],
        'dmet': fb['met_tst_noJVT_et'] - fb['met_tst_et'],
        'ph_pt': ak.firsts(fb['ph_pt']),  # applying the '-150000' shift
        'ph_eta': np.abs(ak.firsts(fb['ph_eta'])),
        'ph_phi': ak.firsts(fb['ph_phi']),
        # 'jet_central_eta': ak.firsts(fb['jet_central_eta']),
        # 'jet_central_pt1': ak.firsts(fb['jet_central_pt']),
        # 'jet_central_pt2': fb['jet_central_pt'][ak.num(fb['jet_central_pt']) >= 2][:, 1],
        # 'jet_central_pt': fb['jet_central_pt'],
        'dphi_met_phterm': np.arccos(np.cos(fb['met_tst_phi'] - fb['met_phterm_phi'])),
        'dphi_met_ph': np.arccos(np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi']))),
        'dphi_met_jetterm': np.where(fb['met_jetterm_et'] != 0,
                                    np.arccos(np.cos(fb['met_tst_phi'] - fb['met_jetterm_phi'])),
                                    0),
        'dphi_phterm_jetterm': np.where(fb['met_jetterm_et'] > 0,
                                        np.arccos(np.cos(fb['met_phterm_phi'] - fb['met_jetterm_phi'])),
                                        4),
        # 'dphi_ph_centraljet1': np.arccos(np.cos(ak.firsts(fb['ph_phi']) - ak.firsts(fb['jet_central_phi']))),
        # 'dphi_ph_jet1': np.arccos(np.cos(ak.firsts(fb['ph_phi']) - ak.firsts(fb['jet_central_phi']))),
        # 'dphi_central_jet1_jet2': np.arccos(np.cos(fb['jet_central_phi'][ak.num(fb['jet_central_phi']) >= 2][:, 0] - fb['jet_central_phi'][ak.num(fb['jet_central_phi']) >= 2][:, 1])),
        'metplusph': fb['met_tst_et'] + ak.firsts(fb['ph_pt']),
        # 'failJVT_jet_pt': fb['failJVT_jet_pt'],
        # 'failJVT_jet_pt1': ak.firsts(fb['failJVT_jet_pt']),
        'softerm': fb['met_softerm_tst_et'],
        'jetterm': fb['met_jetterm_et'],
        'jetterm_sumet': fb['met_jetterm_sumet'],
        'n_jet': fb['n_jet'],
        'n_jet_central': fb['n_jet_central'],
        'n_jet_fwd': fb['n_jet'] - fb['n_jet_central'],
        'vertex': np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) == np.min(np.abs(ak.firsts(fb['pv_truth_z']) - fb['pv_z'])),
        'goodPV': np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5,
        # 'dphi_met_central_jet': np.arccos(np.cos(fb['met_tst_phi'] - ak.firsts(fb['jet_central_phi']))),
        # 'counts': 0.5,
        # 'jet_central_timing1': ak.firsts(fb['jet_central_timing']),
        # 'jet_central_timing': fb['jet_central_timing'],
        # 'jet_central_emfrac': fb['jet_central_emfrac'],
        # 'jet_central_emfrac1': ak.firsts(fb['jet_central_emfrac']),
        'balance': (fb['met_tst_et'] + ak.firsts(fb['ph_pt'])) / np.sum(fb['jet_central_pt']), # need fixing 
        'balance_sumet': (fb['met_tst_et'] + ak.firsts(fb['ph_pt'])) / fb['met_jetterm_sumet'],
        'central_jets_fraction': np.where(fb['n_jet'] > 0, fb['n_jet_central'] / fb['n_jet'], -1),
        'trigger': fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M'],
        # 'dphi_jj': ak.Array([np.arccos(np.cos(phi[1] - phi[0])) if len(phi) > 1 else -1 for phi in fb['jet_central_phi']])
    }

    df = pd.DataFrame(var)

    data = pd.concat([data, df], ignore_index=True)


    # for key, value in var.items():
    #     try:
    #         print(f"{key:30s}: {len(value)}")
    #         print(f"{key:30s}: {value[:5]}")

    #     except Exception as e:
    #         print(f"{key:30s}: error getting length ({e})")

print(data.head())
